#### Setup for file

In [104]:
import pandas as pd
import numpy as np
import streamlit as st
import openai
import time
from bs4 import BeautifulSoup
import requests
import re
import os
import json

from openai._client import OpenAI
from openai import AzureOpenAI


client = AzureOpenAI(
            azure_endpoint="https://da-openai-test.openai.azure.com/",
            api_key="5a90b85ba420469c9b36438e238d70fe",
            api_version="2023-05-15",
        )


In [113]:

today = time.strftime("%m_%d")
today_hour = time.strftime("%m_%d_%H")

# Make a folder for today's data inside the data folder
if not os.path.exists("data/" + today):
    os.mkdir("data/" + today)


In [114]:

# file_path_original_dataframe_csv = "data/all_products_block123.csv"
file_path_original_dataframe_excel = "data/12_15/All Products.xlsx"

# file_path_filter_dataframe_csv = "filter_tags.csv"
file_path_filter_dataframe_excel = "data/12_15/Filter Tags.xlsx"

new_file_path_splitted_dataframe_excel = "data/"+today+"/splitted_tags_" + today_hour + ".xlsx"
new_file_path_splitted_dataframe_csv = "data/"+today+"/splitted_tags_" + today_hour + ".csv"

new_file_path_grouped_dataframe_excel = "data/"+today+"/grouped_tags_" + today_hour + ".xlsx"
new_file_path_grouped_dataframe_csv = "data/"+today+"/grouped_tags_" + today_hour + ".csv"

In [107]:
# # load the dict from pickle file
# import pickle
# with open('./data/url_technical_text_dict.pkl', 'rb') as handle:
#     url_text_dict = pickle.load(handle)

#### Explode the dataframe

In [108]:
df = pd.read_excel(file_path_original_dataframe_excel)
print(len(df))

# Split the 'Product category' column into multiple rows
df = df.assign(
    **{"Product category": df["Product category"].str.split(",")}
).explode("Product category")

# Trim whitespace from the 'Product category' column
df["Product category"] = df["Product category"].str.strip()


print(len(df))



264
444


In [109]:
df

,Unnamed: 0,Product_Name,Product category,Introduction,Block1,Block2,Block3,Features,Technical Specifications,url,Data sheets,downloads,product_family_name,is_range,Hero Image,Other Image
0,0,"Seapath 380 - utilising GPS, GLONASS, Galileo...",Seafloor mapping,"{'heading': 'Introduction', 'text': 'The Seapa...","{'heading': 'Advanced Navigation Integration',...",{'heading': 'Functional Design and System Conf...,"{'heading': 'Product Range and Applications', ...",• 0.007° to 0.02° roll and pitch accuracy depe...,PERFORMANCE POWER SPECIFICATIONS\nHeave accura...,NaN,https://www.kongsberg.com/contentassets/c22a59...,{'Data sheet': ['https://www.kongsberg.com/con...,NaN,NaN,NaN,NaN
0,0,"Seapath 380 - utilising GPS, GLONASS, Galileo...",Inertial solutions,"{'heading': 'Introduction', 'text': 'The Seapa...","{'heading': 'Advanced Navigation Integration',...",{'heading': 'Functional Design and System Conf...,"{'heading': 'Product Range and Applications', ...",• 0.007° to 0.02° roll and pitch accuracy depe...,PERFORMANCE POWER SPECIFICATIONS\nHeave accura...,NaN,https://www.kongsberg.com/contentassets/c22a59...,{'Data sheet': ['https://www.kongsberg.com/con...,NaN,NaN,NaN,NaN
1,1,12 kHz dual-beam transducer,Seafloor mapping,"{'heading': 'Introduction', 'text': 'The 12-16...","{'heading': 'Dual-Beam Functionality', 'text':...","{'heading': 'Beam Characteristics', 'text': 'T...",{'heading': 'Technical and Acoustic Performanc...,Resonant frequency: 12 kHz\nDual beam transduc...,Resonant frequency: 12 kHz\nMaximum transducer...,https://www.kongsberg.com/maritime/products/oc...,https://www.kongsberg.com/contentassets/d106c9...,NaN,Single beam bottom mapping transducers,1.0,['https://www.kongsberg.com/contentassets/e811...,NaN
1,1,12 kHz dual-beam transducer,Geophysical survey,"{'heading': 'Introduction', 'text': 'The 12-16...","{'heading': 'Dual-Beam Functionality', 'text':...","{'heading': 'Beam Characteristics', 'text': 'T...",{'heading': 'Technical and Acoustic Performanc...,Resonant frequency: 12 kHz\nDual beam transduc...,Resonant frequency: 12 kHz\nMaximum transducer...,https://www.kongsberg.com/maritime/products/oc...,https://www.kongsberg.com/contentassets/d106c9...,NaN,Single beam bottom mapping transducers,1.0,['https://www.kongsberg.com/contentassets/e811...,NaN
2,2,120 kHz single-beam transducer,Seafloor mapping,"{'heading': 'Introduction', 'text': 'The 120-2...",{'heading': 'Order Number and Technical Overvi...,{'heading': 'Performance and Construction Deta...,{'heading': 'Acoustic Qualities and Efficiency...,"Resilient Design: Housed in a robust casing, t...",Resonant frequency: 120 kHz Maximum input powe...,https://www.kongsberg.com/maritime/products/oc...,https://www.kongsberg.com/contentassets/d106c9...,NaN,Single beam bottom mapping transducers,1.0,['https://www.kongsberg.com/contentassets/e811...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,259,XPR 100,Navigation & positioning,"{'heading': 'Introduction', 'text': 'The XPR 1...","{'heading': 'Enhanced Operational Capability',...",{'heading': 'Intuitive Operation and Maintenan...,{'heading': 'Complementary Positioning Solutio...,• No moving parts\n• Compact and lightweight d...,XPR 100\n- Operating range: 10 m - 5 km\n- Ran...,https://www.kongsberg.com/maritime/products/ve...,https://www.kongsberg.com/contentassets/ebe8c6...,NaN,MW-based Long-range relative position sensor,1.0,['https://www.kongsberg.com/contentassets/8941...,['https://www.kongsberg.com/contentassets/8941...
260,260,XPR 100 EX,Navigation & positioning,"{'heading': 'Introduction', 'text': 'The XPR 1...",{'heading': 'Long-Range Operation and Visualiz...,{'heading': 'Multiple Sensor Units and Complem...,{'heading': 'Operational Efficiency and Certif...,Key Features: - No moving parts - Compact and ...,XPR 100 EX PERFORMANCE ENVIRONMENTAL SPECIFICA...,https://www.kongsberg.com/maritime/products/ve...,https://www.kongsberg.com/contentassets/ebe8c6...,NaN,MW-based Long-range relativ

#### Make new products_tags

In [110]:
print()

In [125]:
# df.to_csv(new_file_path_splitted_dataframe_csv, index=False)
# df.to_excel(new_file_path_splitted_dataframe_excel, index=False)

In [129]:
# Create a new column to store the tags for each product
# df_tech_new_tags = pd.read_csv("./data/tags_30_11.csv", usecols=["Product_Name", "Product category", "url", "image_url"])
df_tech_new_tags = pd.read_csv(new_file_path_splitted_dataframe_csv)
filter_dataframe = pd.read_excel(file_path_filter_dataframe_excel)

# Set 'category' as the index of filter_dataframe for easier lookup
filter_dataframe.set_index('category', inplace=True)

# Iterate over the rows of the dataframe
for index, row in df_tech_new_tags.iterrows():
    # if pd.isna(row["product_tags"]):
        print(index)
        product_category = str(row["Product category"])  # Convert to string
        # Get the tags for the product category
        if product_category in filter_dataframe.index:
            category_tags = filter_dataframe.loc[product_category].to_dict()  # Use a different variable
            print(row["Product_Name"])
            print(row["Product category"])
            if not pd.isna(row["Data sheets"]):
                print(row["Data sheets"])
            print(category_tags)
            intro_text = str(row["Introduction"]) if pd.notna(row["Introduction"]) else ''
            block1 = str(row["Block1"]) if pd.notna(row["Block1"]) else ''
            block2 = str(row["Block2"]) if pd.notna(row["Block2"]) else ''
            block3 = str(row["Block3"]) if pd.notna(row["Block3"]) else ''
            Features = str(row["Features"]) if pd.notna(row["Features"]) else ''
            Technical_specifications = str(row["Technical Specifications"]) if pd.notna(row["Technical Specifications"]) else ''

            # Concatenate all the text
            website_text = intro_text + block1 + block2 + block3 + Features + Technical_specifications
            messages = [
                {
                    "role": "system",
                    "content": """You are a helpful assistant that generates relevant tags for products. Your responses should be in json format with the top-level category as the key and the tags as the value. For example:
                    
                    {{
                        "Area": "tag", "tag", "tag",
                        "Product Type": "tag", "tag", "tag",
                        "Technology": "tag", "tag", "tag",
                        "Application": "tag", "tag","tag",
                    }}
                    """
                },
                {
                    "role": "user",
                    "content": f"""For the product named '{row['Product_Name']}' in the category ' {row['Product category']} ' with the following description: '{website_text}', please select as many tags that is correct from each top-level category from the provided list: {category_tags}. The top-level categories include 'Area', 'Product Type', 'Technology', and 'Application'. The maximum total number of tags is 15, but you are not required to use all 15. Your response should be a comma-separated list of tags. Please note: avoid using 'tag' as a tag, and do not use the top-level category names as tags. There could be multiple tags from the same top-level category. Return the tags in json format withthe top-level category as the key and the tags as the value. For example:
                    
                    {{
                        "Area": "tag", "tag", "tag",
                        "Product Type": "tag", "tag", "tag",
                        "Technology": "tag", "tag", "tag",
                        "Application": "tag", "tag","tag",
                
                    }}
                    """
                }
            ]
            # response = client.chat.completions.create(
            #     model="gpt-4-1106-preview",
            #     messages=messages
            #     # response_format={ "type": "json_object" }
            # )
            
            response = client.chat.completions.create(
            model="gpt-4-turbo-1106-preview",  # This will correspond to the custom name you chose for your deployment when you deployed a model.
            messages = messages,
            response_format={ "type": "json_object" }
        )

        
            tags = response.choices[0].message.content.strip()
            df_tech_new_tags.loc[index, "product_tags"] = tags
            # print(row["Product_Name"])
            print("______________")
            tags_json = json.loads(tags)
            data = {row["Product category"]: tags_json}
            print(data)
            print("______________")
        else:
            print(f"Category '{product_category}' not found in filter_dataframe")
        df_tech_new_tags.to_csv(new_file_path_splitted_dataframe_csv, index=False)
        df_tech_new_tags.to_excel(new_file_path_splitted_dataframe_excel, index=False)

# save the dataframe as csv
df_tech_new_tags.to_csv(new_file_path_splitted_dataframe_csv, index=False)
df_tech_new_tags.to_excel(new_file_path_splitted_dataframe_excel, index=False)

0
 Seapath 380 - utilising GPS, GLONASS, Galileo, Beidou and QZSS
Seafloor mapping
https://www.kongsberg.com/contentassets/c22a596095db4994a5f26ba216b6b968/110-0017978a_datasheet_seapath380_dec22.pdf
{'Area': 'Underwater', 'product type': 'Echo sounder, Sub-bottom profiler, Sonar', 'technology': 'High-Resolution Sonar, Multibeam Echo Sounder, Single beam Echo sounder, Side-Scan Sonar, Syntethic aperture sonar, Parametric sub-bottom profiler', 'application': 'Deep sea exploration, Oceanography, Subsea survey, Hydrographic survey, Site inspection, Sub-bottom profile survey, Seabed mapping, Seafloor mapping'}
______________
{'Seafloor mapping': {'Area': 'Underwater', 'Product Type': 'Sonar', 'Technology': 'Multibeam Echo Sounder', 'Application': 'Hydrographic survey, Seabed mapping, Seafloor mapping'}}
______________
1
 Seapath 380 - utilising GPS, GLONASS, Galileo, Beidou and QZSS
Inertial solutions
https://www.kongsberg.com/contentassets/c22a596095db4994a5f26ba216b6b968/110-0017978a_dat

JSONDecodeError: Expecting ':' delimiter: line 5 column 58 (char 167)

#### Groupe the dataframe

In [ ]:
import ast
def extract_tags(tags_str):
    tags_obj = ast.literal_eval(tags_str)
    if isinstance(tags_obj, dict):
        tags_values = tags_obj.values()
    elif isinstance(tags_obj, tuple):
        tags_values = tags_obj
    else:
        raise ValueError(f"Unexpected type {type(tags_obj)}: {tags_obj}")
    
    tags_list = []
    for value in tags_values:
        if isinstance(value, list):
            tags_list.append(', '.join(map(str.strip, value)))
        elif isinstance(value, dict):
            tags_list.append(str(value))
        else:
            tags_list.append(value.strip())
    return ', '.join(tags_list)

In [ ]:
df_with_many_tags = pd.read_csv(new_file_path_splitted_dataframe_csv)
print(len(df_with_many_tags))
# df_with_many_tags['product_tags'] = df_with_many_tags['product_tags'].apply(extract_tags)
# combine all the tags with same product_name into one row. Let the name be the same as the first row, but for tags, combine them into two lists of tags. For the category, combine them into a list of categories. For website_url, image_url, choose the first one. 



df_with_many_tags = df_with_many_tags.groupby('Product_Name').agg({
    'product_tags': lambda x: ', '.join(set(map(lambda y: str(y).strip(), x))),
    'Product category': lambda x: ', '.join(map(lambda y: str(y).strip(), x)),
    'url': 'first',
    'Data sheets': 'first',
    'Introduction': 'first',
    'Block1': 'first',
    'Block2': 'first',
    'Block3': 'first',
    'Features': 'first',
    'Technical Specifications': 'first',
    'downloads': 'first',
    'product_family_name': 'first',
    'is_range': 'first',
    'Hero Image': 'first',

    
    


    
}).reset_index()
print(len(df_with_many_tags))

# save to excel
df_with_many_tags.to_excel(new_file_path_grouped_dataframe_excel, index=False)
df_with_many_tags.to_csv(new_file_path_grouped_dataframe_csv, index=False)

444
262


#### Merge with software tags

In [ ]:
# # if dataframe do not have column "is_software" then merge the column from software dataframe to the dataframe on the product name column
# df_grouped_tags = pd.read_csv(new_file_path_grouped_dataframe_csv)
# #check if the column is_software exists
# if "is_software" not in df_grouped_tags.columns:
#     # read the software dataframe
#     df_software = pd.read_csv("data/tags_27_11.csv")
#     # merge the column is_software to the dataframe on the product name column
#     df_grouped_tags = pd.merge(df_grouped_tags, df_software[["Product_Name", "is_software"]], on="Product_Name", how="left")
#     # save the dataframe
#     print(df_grouped_tags.head())

                                        Product_Name  \
0                        AIS physical shore stations   
1                      AIS service management system   
2                                AIS space receivers   
3  APOS - Operating station for HiPAP and HPR Aco...   
4  APOS Survey - Surveyor's independent Operator ...   

                                        product_tags  \
0  AIS transponder, Autonomous Navigation, Underw...   
1  Acoustic Control System, AIS transponder, Mari...   
2  Transceiver, Radio, reconfigurable SDR, VDES t...   
3  Navigation system, Positioning system, Acousti...   
4  Navigation system, Positioning system, Acousti...   

                      Product category  \
0            Surveillance & monitoring   
1            Surveillance & monitoring   
2              Communication solutions   
3  Underwater navigation & positioning   
4  Underwater navigation & positioning   

                                                 url  \
0  https://www.ko